# Fine-tuning de DistilBERT para detección de hate speech

En este notebook vamos a entrenar un modelo basado en transformers (DistilBERT)
para clasificar comentarios de YouTube como **tóxicos (1)** o **no tóxicos (0)**.

Objetivos:

- Cargar el dataset preprocesado (`text_basic` + `IsToxic`).
- Preparar los datos para HuggingFace (tokenizer + `Dataset`).
- Fine-tuning de `distilbert-base-uncased` para clasificación binaria.
- Evaluar el modelo en el conjunto de test (accuracy, precision, recall, F1, ROC-AUC).
- Guardar el modelo en `backend/models/distilbert_toxic_v1`.
- Guardar las métricas en `data/results/distilbert_toxic_v1.json`.


## 1. Configuración inicial y rutas del proyecto

En esta sección definimos las rutas relativas dentro del repositorio y hacemos
las importaciones básicas que necesitaremos más adelante.


In [2]:
import os
from pathlib import Path

# Ruta del repo (este notebook está en backend/notebooks/)
NOTEBOOK_DIR = Path.cwd()
BACKEND_DIR = NOTEBOOK_DIR.parent           # backend/
ROOT_DIR = BACKEND_DIR.parent               # raíz del proyecto
DATA_DIR = ROOT_DIR / "data"
PREPROC_DIR = DATA_DIR / "preprocessing_data"
RESULTS_DIR = DATA_DIR / "results"
MODELS_DIR = BACKEND_DIR / "models"

# Ficheros concretos
CSV_PATH = PREPROC_DIR / "youtoxic_english_1000_clean.csv"
METRICS_JSON_PATH = RESULTS_DIR / "distilbert_toxic_v1.json"
DISTILBERT_MODEL_DIR = MODELS_DIR / "distilbert_toxic_v1"

print("Notebook dir:", NOTEBOOK_DIR)
print("CSV path:", CSV_PATH)
print("Metrics JSON:", METRICS_JSON_PATH)
print("Model dir:", DISTILBERT_MODEL_DIR)


Notebook dir: c:\Users\yeder\Documents\Factoria F5 Bootcamp IA\Proyecto_X_NLP_G4\backend\notebooks
CSV path: c:\Users\yeder\Documents\Factoria F5 Bootcamp IA\Proyecto_X_NLP_G4\data\preprocessing_data\youtoxic_english_1000_clean.csv
Metrics JSON: c:\Users\yeder\Documents\Factoria F5 Bootcamp IA\Proyecto_X_NLP_G4\data\results\distilbert_toxic_v1.json
Model dir: c:\Users\yeder\Documents\Factoria F5 Bootcamp IA\Proyecto_X_NLP_G4\backend\models\distilbert_toxic_v1


## 2. Instalación de dependencias

Instalamos las librerías necesarias para trabajar con transformers y los
datasets de HuggingFace. Esta celda solo es necesario ejecutarla la primera vez
(en el entorno del proyecto).


In [2]:
%pip install -q "transformers>=4.40.0" "datasets>=2.19.0" "evaluate" "accelerate" "scikit-learn"


Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -q "tf_keras"

Note: you may need to restart the kernel to use updated packages.


## 3. Imports principales

Importamos las librerías de HuggingFace, sklearn y utilidades varias.


In [3]:
import numpy as np
import pandas as pd
import torch

from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
import evaluate
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
)


## 4. Carga del dataset preprocesado

Cargamos el CSV que generamos en el notebook de preprocesado.  
Suponemos que tiene al menos estas columnas:

- `text_basic`: texto preparado para modelos modernos.
- `IsToxic`: etiqueta binaria (0 = no tóxico, 1 = tóxico).


In [4]:
df = pd.read_csv(CSV_PATH)

print(df.head())
print("\nColumnas:", df.columns.tolist())
print("\nDistribución de IsToxic:")
print(df["IsToxic"].value_counts(normalize=True))


              CommentId      VideoId  \
0  Ugg2KwwX0V8-aXgCoAEC  04kJtp6pVXI   
1  Ugg2s5AzSPioEXgCoAEC  04kJtp6pVXI   
2  Ugg3dWTOxryFfHgCoAEC  04kJtp6pVXI   
3  Ugg7Gd006w1MPngCoAEC  04kJtp6pVXI   
4  Ugg8FfTbbNF8IngCoAEC  04kJtp6pVXI   

                                                Text  IsToxic  IsAbusive  \
0  If only people would just take a step back and...    False      False   
1  Law enforcement is not trained to shoot to app...     True       True   
2  \r\nDont you reckon them 'black lives matter' ...     True       True   
3  There are a very large number of people who do...    False      False   
4  The Arab dude is absolutely right, he should h...    False      False   

   IsThreat  IsProvocative  IsObscene  IsHatespeech  IsRacist  \
0     False          False      False         False     False   
1     False          False      False         False     False   
2     False          False       True         False     False   
3     False          False      False     

### 4.1 Limpieza ligera y renombrado de la columna label

HuggingFace `Trainer` espera normalmente una columna `labels`.  
Renombramos `IsToxic` → `label` y nos aseguramos de que sea un entero.


In [5]:
# Eliminamos filas con textos o etiquetas nulas por seguridad
df = df.dropna(subset=["text_basic", "IsToxic"]).reset_index(drop=True)

# Renombrar la columna de target a 'label'
df = df.rename(columns={"IsToxic": "label"})
df["label"] = df["label"].astype(int)

df[["text_basic", "label"]].head()


,text_basic,label
0,If only people would just take a step back and...,0
1,Law enforcement is not trained to shoot to app...,1
2,Dont you reckon them 'black lives matter' bann...,1
3,There are a very large number of people who do...,0
4,"The Arab dude is absolutely right, he should h...",0
